In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import Libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # regex pattern matching
import warnings
warnings.filterwarnings('ignore')# Ignoring warnings

# **Reading Dataset**

In [ ]:
zmt=pd.read_csv('/kaggle/input/tomato-reviews/zomato.csv')

In [ ]:
zmt.head() # reading 1st five rows of our dataset using .head()

In [ ]:
zmt.info() #checking information of our dataset

Here we can see we are having 17 columns and some columns including NaN values and incorrect data_types.

Ex: approx_cost(for two people) column has integer values but having object data_type.

# **Dropping unnecessary columns**

Unnecessary columns are those columns which are not that useful for analysis.

Ex: Phone_Number, we not able to predict anything using Phone numbers. So, we can drop that columns which we are not going to use.

In [ ]:
zmt.drop(['url','reviews_list','menu_item','address','phone','dish_liked'],axis=1,inplace=True)

In [ ]:
zmt.head(2)

Now we will get only those columns which are useful for analysis.

# **Renaming Columns**

We are renaming the columns for better readability

In [ ]:
zmt.rename(columns={'name':'restaurants','rate':'rating','cuisines':'food_type','listed_in(type)':'type','listed_in(city)':'city','approx_cost(for two people)':'cost'},inplace=True)

In [ ]:
zmt.head(1)

# **Dropping NAN values**

In [ ]:
zmt.dropna(inplace = True) # Droping NaN values for better result using dropna()

# **Cleaning individual Columns**

**Column: Restaurant**

Column contains restaurant names


In [ ]:
zmt.groupby('restaurants').count().head() # we are grouping the restaurants to see the restaurant names and their count.

Here we can see we are got restaurant names having disturbed characters

so we need to remove that characters from the name using pattern matcing to get orignal names.

In [ ]:
# '[Ãx][^A-Za-z]+' pattern I got to find disturbed characters using regex.And Replacing those charachters with empty space. 
zmt['restaurants']=zmt['restaurants'].str.replace('[Ãx][^A-Za-z]+','',regex=True)

In [ ]:
zmt.groupby('restaurants').count().head()

Here see we successfully removed the disturbed characters from restaurant name(refer 5th row to see transformation)

By reading the databse I found the names having incorrect spell. Let me correct them as well.

In [ ]:
zmt['restaurants']=zmt['restaurants'].str.replace('Caf-|Caf ','Cafe ',regex=True) # replacing the Caf- or Caf names with cafe using pattern matching

**Column: order**

Column shows that restaurants accepting online orders or not.

In [ ]:
zmt['online_order'].unique() # Using .unique() we can find the unique values from column

Here, we can see only Yes and No values are present so there is no need to clean the column.

**Column: Book_table**

Column shows that restaurants having table booking facility or not.

In [ ]:
zmt['book_table'].unique()

Here also we got the same result so no need to perform any cleaning

**Column: rating**

Column shows the rating of the hotel out of 5

In [ ]:
zmt['rating'].unique()

Here, we can see rating column having string values and having object datatype including 'NEW' and '-' values. we know that rating is always out of 5 and in decimal format. So, we need to clean our 'rating' columns.

Need changes:

Remove 'NEW' and '-' values.
Remove '/5' from the rating.
Covert datatype object to float (float_datatype has decimal values).


In [ ]:
replace=lambda x:x.replace('/5',"") # lambda function to replace /5 to empty string
l=[] # list to store cleaned values
for val in map(replace, zmt['rating']): # map function to read data from column and replace /5 to empty string
    if val!='NEW' and val!='-':  # excluding 'NEW' and '-' values
        var=float(val) # converting the result in float datatype and storing into one variable
    l.append(var) # appending cleaned values in created list
zmt['rating']=l  # updating rating column with new and cleaned values

In [ ]:
zmt['rating'].unique(),zmt['rating'].dtype

we got the cleaned values with float datatype.

**Columns: vote**

Column contains number of votes hotel got.

In [ ]:
zmt['votes'].isnull().value_counts() # checking column having null values or not using .isnull()

Here, we can see there is no null values and having correct datatype. So, no need to clean anything.

**Column: Location**

Column contains locations of restaurants.



In [ ]:
zmt.location.unique() # This syntax works similar like previous so don't be confuse.

Here, aslo all things are correct so no need to perform data cleaning

**Column: Rest_type and Food_type**

Columns shows that the restaurant types and which type of food restaurants have.

In [ ]:
zmt.rest_type.unique(), zmt.rest_type.isnull().value_counts()

In [ ]:
zmt.food_type.unique(), zmt.food_type.isnull().value_counts()

As we see column 'rest_type' and 'food_type' are aslo correct so no need to perform any cleaning operations.

Column: Cost

In [ ]:
zmt.cost.unique()

Here see cost column string values with ',' values. And we know cost is always in int sp we need to make some transformations.

Changes need:

Remove ',' from values
Change datatype object to integer(int)

In [ ]:
zmt['cost']= zmt['cost'].apply(lambda x:x.replace(",","")).astype(int) # we can also apply lambda function using .apply()

In [ ]:
zmt.cost.unique(),zmt.cost.dtype

Here, Using lambda function we replaced ',' to empty string and using astype(int) we converted datatype object to int and update in the cost column and we got cleaned data.

**Column: type**

Column shows which types of arrangment restaurant have.

In [ ]:
zmt.type.unique()

Here also all values are correct and having correct datatype so no need to perform cleaning.

**Column: City**

Column contains name of cities of restaurants

In [ ]:
zmt.city.unique()

Here also all values are correct and having correct datatype so no need to perform cleaning.

# **Dropping Duplicates**

In [ ]:
zmt.duplicated().value_counts() # by using .duplicated() we can see duplicate values from our dataset.

We got 80 duplicate values in our dataset we need to remove those.

In [ ]:
zmt.drop_duplicates(keep='last',inplace=True)# dropping duplicates values using .drop_duplicates()
zmt.reset_index(drop=True,inplace=True) # reseting index

In [ ]:
zmt.duplicated().value_counts()

As a final step we removed all duplicate values from our dataset and we got fully cleaned data.

In [ ]:
zmt

In [ ]:
zmt.info()

In [ ]:
zmt.to_csv('./zomato_clean_data.csv') # Saivng cleaned dataset in output directory.

# **Data Visualization**

In data visualization we going to visualize our data how it is, and trying to find some informative data from our dataset.

Here we are going to use two python libraries For visualization.

Seaborn
Matplotlib
So, first we are going to import them.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

So we know that we are working on zomato or we can say Restaurants rating dataset. By seeing a dataset we can visualize more things,
Ex:
We can find, Restaurants Delivering online or not.
Restaurants allowing table booking or not.
Best location to visit.
Types of Restaurants. ##### and many more.

**How many restaurants accepting online order from zomato**

In [ ]:
zmt.head(1) # see the name of the cloumns 

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='online_order',data=zmt) # countplot gives us count of the values
plt.show()

By observing the graph, we can say that 25000+ restaurants are accepting online order for zomato and 15000 restaurants not accepting online orders.

**Find the best locatiob by seeing the dataset**

How can we find? :-

We have two columns 'rating' and 'votes' so by getting the average of those columns with repect to 'location' we can find the best location.

In [ ]:
zmt.head(1)

In [ ]:
# sns.barplot(x='location',y='votes',hue='online_order',data=zmt[:50])

In [ ]:
plt.figure(figsize=(16,10)) # seeting graph size using matplotlib
ax = plt.subplot(2,1,1)   # use of Subploting
loc_rating=zmt.groupby('location').agg({'rating':'median'})
rating_sorted_loc=loc_rating.sort_values('rating',ascending=False).head(5).reset_index() # sorting location on avg of votes in descending order and reading first 5 values
sns.barplot(x='location',y='rating',data=rating_sorted_loc) # ploting graph using seaborn

ax = plt.subplot(2,1,2)
loc_votes=zmt.groupby('location').agg({'votes':'median'})
votes_sorted_loc=loc_votes.sort_values('votes',ascending=False).head(5).reset_index() # sorting location on avg of rating in descending order and reading first 5 values
sns.barplot(x='location',y='votes',
            data=votes_sorted_loc)
plt.show()

By observing the above graphs,'levelle Road' has an high rating and votes as well as compare to other.

So, we can say that 'levelle Road' is a best location by comparing 'votes' and 'rating' of locations.

**Find types of restaurant and their count**

In [ ]:
zmt.head(1)

In [ ]:
plt.figure(figsize=(10,20))
rest_types=zmt.groupby('rest_type')['restaurants'].count().reset_index()# grouping restaurants types and getting 'restaurant' count
sorted_rest_types=rest_types.sort_values('restaurants',ascending=False).head(40) # sorting restaurant types on 'restaurants' counts in descending order and reading 40 values
rest_types.rest_type.count(),sns.barplot(x='restaurants',y='rest_type',data=sorted_rest_types,orient='h')# drawing horizontal bar plot to see results

by observing above graph, the 1st line of grapth shows the number of rest_type we have that is 87 and we plotted only top 40 types which are high count of restaurants types.

ex:

14000 Quick Bites restaurants.

10000 Casual Dining restaurants.

and so on...

**Find Costs of restaurants**

In [ ]:
zmt.head(1)

In [ ]:
plt.figure(figsize=(10,5))
sns.kdeplot(data=zmt,x='cost',)# KDE plot 
plt.show()


By observing above kde plot we can say that most of the restaurants have cost between 1 to 1000Rs. for food and remaining have above 1000Rs for their food.

**Restaurants having table booking facility**

Here, we can use countplot but try to drow pointplot to see how it looks like.

In [ ]:
plt.figure(figsize=(10,5))
table_booking= zmt.groupby('book_table')['restaurants'].count().reset_index()
sns.pointplot(x='book_table',y='restaurants',color='b',data=table_booking)
plt.show()

By seeing above pointplot we can say that only 7000 restaurants have table booking facility and 35000+ restaurants don't have facility of table booking.

**Find number of restaurants at following locations:**

'BTM'
'Basavanagudi'
'West Bangalore'
'Whitefield'
'Yeshwantpur'

In [ ]:
df=pd.DataFrame(zmt.groupby('location')['restaurants'].count()).reset_index()
criteria=df['location'].isin(['BTM','Basavanagudi','West Bangalore','Whitefield','Yeshwantpur']) # selecting given locations from dataset using isin()
plt.figure(figsize=(10,5)) 
sns.barplot(x='location',y='restaurants',data=df[criteria])
plt.show()


By seeing given plot, we can notice that how many restaurants present are at given locations.

Ex:-

BTM lication has 4000+ restaurants

**Find most famous restraunt chain having most franchises**

We working on a zomato dataset of bangalore location so we can asume that all locations are belongs to bangalore.

In this problem we need to find restaurant at each location having more than 1 branch and high rating.

In [ ]:
df1=pd.DataFrame(zmt.groupby(['location','restaurants','rating']).count()).reset_index() # getting location,restraunts and rating and their count of restaurants
df1

In [ ]:
sns.distplot(df1['cost'])

Note that all columns having some count values these are nothing but a count of that restaurant at given location.

We can see some restaurants having 1 count and some having more than 1. We want that restaurants which having more than 1 count and high rating.

In [ ]:
chains_restaurants=df1[(df1['book_table']>1)]
famous_restaurants=pd.DataFrame(chains_restaurants.groupby('location')[['restaurants','rating']].max()).reset_index()
famous_restaurants

Finally, we got restaurant at each location has more that one branch and highest rating. Total 89 result we got. Let's try to plot only first 5 to get idea.

In [ ]:
sns.catplot(x="location", y="rating", hue="restaurants",kind='bar',height=7,data=famous_restaurants.head(5))

Here we can see that famous restaurant having more that one branch at location and their rating as well.

Ex.:

We can say that, at BTM location in bangalore "eat.fit" restaurant is a famous restaurant franchise having maximum rating(4.9) compare to other restaurants franchise's at same location.

At Banashankari location in bangalore "Yo Roll Corner" restaurant is a famous restaurant franchise having maximum rating(4.6) compare to other restaurants franchise's at same location.

At Banaswadi location in bangalore "Zam Zam Restaurant" restaurant is a famous restaurant franchise having maximum rating(4.0) compare to other restaurants franchise's at same location.
and so on...

**Find how many voters gives rating for each 'type' and aggregate rating of that 'type'**

In [ ]:
df2=zmt.groupby('type').agg({'votes':'sum','rating':'mean'}).nlargest(7,['votes']).reset_index()
sns.catplot(x='type',y='rating',hue='votes',kind='bar',height=8,data=df2)
plt.show()

Here we can see that, how many voters gives rating for each type and average rating of that type.

Ex.:

Most of the voters(Total 5235825 voters) gives rating to the Delivery and its average rating is 3.7.

and so on...

Note:
If we see delivery type has low rating but number of voters are high. So it is ok to have low rating because of,

If one voter gives rating to a some product that time we have possibility that the product having 5/5 rating.

And if number of voters are increases then it is possible to get low rating depending upon how many voters rated to that product.

**Gaussian rest type (Normal Distribution) of rating**

In [ ]:
sns.distplot(zmt['rating'])

We got a normal distribution form of our rating and we observe that we have maximum ratings between 3.5 to 4.5.

**Find how many restaurants have Chinese and North Indian food in their food type**

In [ ]:
Chinese=len([i for i in zmt['food_type'] if 'Chinese' in i]) # count of restaurants who serving chinese food.
North_India=len([i for i in zmt['food_type'] if 'North India' in i]) # count of restaurants who serving North Indian food.
Restaurant_count=[Chinese,North_India] #creating list
Food_Type=['Chinese','North India']
df3 = pd.DataFrame({'Food_Type':Food_Type,'Restaurant_count':Restaurant_count})#.set_index('Food_Type')# creating dataset of extracted data

In [ ]:
sns.barplot(x='Food_Type',y='Restaurant_count',data=df3) # countplot gives us count of the values
plt.show()

Here, We got actual count of restaurants who serving chinese and North Indian food.

**Find the most profitable type of restaurant**

In [ ]:
df4=zmt.groupby('type').agg({'cost':'mean'})
df4.cost.plot(kind='pie',autopct='%1.1f%%',figsize=(9,9),shadow=True)
plt.show()

Here, by seeing the total percentage of average cost we can observe that. The restaurants having Drink and Nightlife facility are in high profit than other types.

# Conclusion****

In the Given dataset we have explored and learned many things,

* we learned about how to clean our data.
* How we can interprete data by visualizing it.